# 🧠 **Simplified LLM Dissection Lab: See Inside and Understand the LLM Brain** 🧠
### For Teachers/Admins:
##### It's suggested to run Step 0, 1, 1.5, & 2 shortly before a class session starts to reduce loading times if possible.
##### Make sure that *H100, A10, or V6e-1 TPU* are chosen if possible. *H100* being the best for this project.
##### Read the main README.md for more support.

# 🔍 Step 0 (Recommended): Check Hardware 🔍
### Run this cell to see what hardware Google Colab has assigned to you.
### Generate a "Safety Report" recommending which models you can run without crashing.
### If you get a message that says you're using a GPU *(H100, A10, or V6e-1 ideally)*, that is good.

In [ ]:
import torch, psutil, os

def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*50)
print("          HARDWARE REPORT")
print("="*50)

# System RAM Check
total_ram = psutil.virtual_memory().total
print(f"System RAM: {get_size(total_ram)}")

# GPU Check
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_total = torch.cuda.get_device_properties(0).total_memory

    print(f"✅ Great, you are in a runtime that has a ({gpu_name}). ✅")
    print(f"GPU VRAM: {get_size(vram_total)}")
    print("\n|----------------------------- CAPABILITY EXAMPLE --------------------------------|")
    print("• A100 / H100:   High-End. Safe to run pretty much anything in this project.")
    print("• V6e / V5e TPU: High-End. Likely safe to run pretty much anything in this project.")
    print("• L4 / A10:      Mid-Range. Safe for models up to 40B+ parameters most likely.")
    print("• T4:            Stick to models up to and or around DeepSeek-Lite. (16B+ params).")
    print("• 2080 Ti:       Can run up to DeepSeek-Lite.")
    print("• NOTE:          Llama-4-Scout might not be runnable.")
    print("|-----------------------------------------------------------------------------------|")

# CPU Fallback
else:
    print("❌❌❌❌❌ NO GPU DETECTED (Running on CPU) ❌❌❌❌❌")
    print("-" * 50)
    print("⚠️⚠️⚠️ PERFORMANCE WARNING: ⚠️⚠️⚠️")
    print("   • You should only run the SMALLER OR TINY models only.")
    print("   • Safe to run: GPT-2, TinyLlama, ")
    print("   • ⛔ AVOID any model larger than 4B parameters.")
    print("\n💡 TIP: Go to 'Runtime' -> 'Change Runtime Type' > Select some 'GPU' for better results, like A100.")

print("="*50)

# 🚀 Step 1: Install & Restart (Run this, wait for the crash, then go to Cell 2) 🚀

In [ ]:
import os, sys, time

repo_name = "Simplified-How-LLMs-Work-Visualized"
if not os.path.exists(repo_name):
    print("📂 Downloading Resources From Evan's GitHub... 📂")
    os.system(f'git clone https://github.com/evanfarnping/{repo_name}.git')

if os.path.exists(repo_name):
    os.chdir(repo_name)

if not os.path.exists('.setup_complete'):
    print("⏳ Installing libraries (NumPy, Torch, etc)... This may take 1-5 minutes. ⏳")
    sys.stdout.flush()
    os.system('pip install -q -r requirements.txt')
    
    with open('.setup_complete', 'w') as f:
        f.write('done')
    
    print("\n INSTALLATION COMPLETE.")
    print("The Runtime will now RESTART automatically to apply changes.")
    print("✅⚠️⚠️⚠️✅ WAIT FOR AND IGNORE THE 'Your session crashed for an unknown reason.' POPUP. ✅⚠️⚠️⚠️✅")

    sys.stdout.flush()
    time.sleep(7.5)
    print("👉 Once popup finishes, move to Cell 2 👉")
    os.kill(os.getpid(), 9)
else:
    print("✅ Requirements already installed. You can proceed to Cell 2. ✅")

# Step 1.5: Admin Pre-Load (OPTIONAL)
### Run this cell to download common models to the disk cache immediately.
### This saves time during class. This will NOT load them into RAM. But be careful about your storage.

In [ ]:
import os, sys

# Same token logic in the later step.
part_1 = "hf"
part_2 = "_iKEQoqWclmnlpBwbud"
part_3 = "emYXZcHAqesgsszm"
CLASS_TOKEN = part_1 + part_2 + part_3

# Define the models we want to pre-cache in hard storage.
PRE_LOAD_TARGETS = {
    "GPT_2": "gpt2",
    "Pythia_160M": "EleutherAI/pythia-160m",
    "Qwen2.5_0.5B": "Qwen/Qwen2.5-0.5B-Instruct",
    "TinyLlama_1.1B": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "Qwen3_1.7B": "Qwen/Qwen3-1.7B",
    "Phi4Mini_4B": "microsoft/Phi-4-mini-instruct",
    "Mistral_7B": "mistralai/Mistral-7B-Instruct-v0.3",
    "Qwen2.5_14B": "Qwen/Qwen2.5-14B-Instruct",
    "DeepSeek_Lite": "deepseek-ai/DeepSeek-V2-Lite-Chat"
}

# Select which models to pre-download:
#@markdown Very small, and used in some of the scenarios.
Download_GPT_2 = True # @param {type:"boolean"}
#@markdown Very small, very similar to GPT-2 effectively. Can probably not pre-cache.
Download_Pythia_160M = False # @param {type:"boolean"}
#@markdown A tiny reasoning model. It's mini and easy to show thinking tokens.
Download_Qwen2_500M = True # @param {type:"boolean"}
#@markdown A great model to show general chat features while being very small.
Download_TinyLlama_1B = True # @param {type:"boolean"}
#@markdown Next step for a small reasoning model. Better thinking tokens.
Download_Qwen3_2B = True # @param {type:"boolean"}
#@markdown Overall pick in terms of general chat while being small. Good for nearly any experiment.
Download_Phi_4_mini_4B = True # @param {type:"boolean"}
#@markdown Entering the "smart" general model territory. Better than Phi for the most part. 
Download_Mistral_7B = True # @param {type:"boolean"}
#@markdown Smarter, but takes more space. Keep off unless you know you'll use it over DeepSeek or have extra space.
Download_Qwen2_14B = False # @param {type:"boolean"}
#@markdown Good reasoning model for the size. In this project it's likely the slowest; pre-cache will help with speed.
Download_DeepSeek_Lite = True # @param {type:"boolean"}

selected_repos = []
if Download_GPT_2: selected_repos.append(PRE_LOAD_TARGETS["GPT_2"])
if Download_Pythia_160M: selected_repos.append(PRE_LOAD_TARGETS["Pythia_160M"])
if Download_Qwen2_500M: selected_repos.append(PRE_LOAD_TARGETS["Qwen2.5_0.5B"])
if Download_TinyLlama_1B: selected_repos.append(PRE_LOAD_TARGETS["TinyLlama_1.1B"])
if Download_Qwen3_2B: selected_repos.append(PRE_LOAD_TARGETS["Qwen3_1.7B"])
if Download_Phi_4_mini_4B: selected_repos.append(PRE_LOAD_TARGETS["Phi4Mini_4B"])
if Download_Mistral_7B: selected_repos.append(PRE_LOAD_TARGETS["Mistral_7B"])
if Download_Qwen2_14B: selected_repos.append(PRE_LOAD_TARGETS["Qwen2.5_14B"])
if Download_DeepSeek_Lite: selected_repos.append(PRE_LOAD_TARGETS["DeepSeek_Lite"])

def admin_download():
    # Check if libraries are installed from Step 1
    try:
        from huggingface_hub import snapshot_download, login
    except ImportError:
        print("❌❌❌❌❌ CRITICAL ERROR: Libraries not found. ❌❌❌❌❌")
        print("Run 'Step 1: Install & Restart' before running this cell.")
        return

    print("🔑 Authenticating for better download speed... 🔑")
    try:
        if len(CLASS_TOKEN) > 7:
            login(token=CLASS_TOKEN, add_to_git_credential=True)
            print("Success! Logged in.")
        else:
            print("⚠️ No token provided. Downloading as Anonymous (Slower).")
    except Exception as e:
        print(f"⚠️ Login Warning: {e}")

    print(f"📦 Starting Admin Download for {len(selected_repos)} models... 📦")
    print("Note: This downloads files to the disk cache. It doesn't consume RAM.")
    
    for repo in selected_repos:
        print(f"\n⬇️ Downloading: {repo}... ⬇️")
        try:
            snapshot_download(
                repo_id=repo, 
                allow_patterns=["*.safetensors", "*.json", "*.model", "*.txt", "*.bin"]
            )
            print(f"✅ Cached: {repo} ✅")
        except Exception as e:
            print(f"⚠️ Failed to download {repo}: {e} ⚠️")

    print("\n✅✅✅✅✅ PRE-LOAD COMPLETE ✅✅✅✅✅")
    print("Students can now run experiments using these models instantly.")

if __name__ == "__main__":
    admin_download()

# 🔑 Step 2: Login & Load Engine (Run this after Step 1 finishes) 🔑

In [ ]:
import sys, os

# --- TEACHER CONFIGURATION (PASTE TOKEN IN 3 PARTS HERE) --- #
# You can get a token at https://huggingface.co/settings/tokens 
# These are free tokens, not associated with any paid plan.
# If Rate Limit occurs, try other tokens. 
# Uncomment this code then delete old token:
# part_1 = hf
# part_2 = _pmMlbqyWgQpPOYFMKS
# part_3 = lnRFWnZdZFMDghRY
# CLASS_TOKEN = part_1 + part_2 + part_3
# part_1 = hf
# part_2 = _iKEQoqWclmnlpBwbud
# part_3 = emYXZcHAqesgsszm
# CLASS_TOKEN = part_1 + part_2 + part_3

part_1 = "hf"
part_2 = "_NohyiEXNSIjeCkeyPD"
part_3 = "LHCIilWruzzseeLU"
CLASS_TOKEN = part_1 + part_2 + part_3 # TO BYPASS KEY DETECTION
# ----------------------------------------------------------- #

if os.path.basename(os.getcwd()) != "Simplified-How-LLMs-Work-Visualized":
    if os.path.exists("Simplified-How-LLMs-Work-Visualized"):
        os.chdir("Simplified-How-LLMs-Work-Visualized")

sys.path.append(os.path.abspath("main_configs"))
sys.path.append(os.path.abspath("src"))

print("🔧 Applying System Configurations... 🔧")
video_script = "src/make_comparison_video.py"
if os.path.exists(video_script):
    with open(video_script, "r") as f: code = f.read()
    code = code.replace("figsize=(18, fig_height)", "figsize=(14, fig_height)")
    code = code.replace("dpi=120", "dpi=100")
    with open(video_script, "w") as f: f.write(code)

main_script = "main_configs/main.py"
if os.path.exists(main_script):
    with open(main_script, "r") as f: main_code = f.read()
    if "import threading" not in main_code:
        main_code = "import threading\n" + main_code
    with open(main_script, "w") as f: f.write(main_code)

# Login
print("🔑 Authenticating with Hugging Face... 🔑")
from huggingface_hub import login
try:
    if len(CLASS_TOKEN) > 7:
        login(token=CLASS_TOKEN, add_to_git_credential=True)
        print("✅ Success! Logged in. ✅")
    else:
        print("⚠️⚠️⚠️ No token provided. ⚠️⚠️⚠️")
except Exception as e:
    print(f"⚠️ Login Warning: {e} ⚠️")

# Load Src Engine Code 
print("🚀 Loading AI Engine... 🚀")
try:
    import main as engine
    print("✅✅✅ ENGINE READY! Proceed to Cell 3. ✅✅✅")
except ImportError:
    print("❌❌❌❌❌ Critical Error: Libraries not found. Did Cell 1 run and restart? ❌❌❌❌❌")

# 🧪 Step 3: Run Experiments 🧪
### Select an experiment from the list and click Run.

In [ ]:
import sys, os, glob, time
import matplotlib.pyplot as plt
from IPython.display import Image, Video, display

# NOTE: Some scenarios suggest changing values once you run them to compare different models. Please read more information about the scenario and what to change in the "scenarios_to_try" directory/folder.
#@markdown Choose a pre-built experiment/scenario to analyze and reflect on. Learn more in the "scenarios_to_try" directory/folder.
Scenario = "spelling_and_structure_matter!" # @param ["medical_bias", "safety_overrides", "server_prompting", "bad_prompts_vs._good_prompts", "knowledge_cutoff", "simple_vs._complex_models", "thinking_vs._chat_models", "bias_in_roles", "fake_empathy_vs._logic", "chinese_vs._USA_data","LLMs_suck", "negative_vs._positive_AIs", "raw_LLMS_vs._chatbot_LLMs","spelling_and_structure_matter!"]

if 'engine' not in locals():
    sys.path.append(os.path.abspath("main_configs"))
    sys.path.append(os.path.abspath("src"))
    import main as engine

start_time = time.time()
script_path = f"scenarios_to_try/{Scenario}/scenario.py"

if os.path.exists(script_path):
    print(f"⚡ Running Experiment: {Scenario} ⚡")
    print("Please wait while the model loads...")
    
    !python "{script_path}"

    plt.close('all')

    print("\n--- RESULTS ---")
    videos = glob.glob(f"scenarios_to_try/{Scenario}/*.mp4")
    images = glob.glob(f"scenarios_to_try/{Scenario}/*.png")

    new_videos = [v for v in videos if os.path.getmtime(v) > start_time]
    new_images = [i for i in images if os.path.getmtime(i) > start_time]

    if new_videos:
        for vid in new_videos:
            display(Video(vid, embed=True, width=825))
    elif new_images:
        for img in new_images:
            display(Image(filename=img, width=825))
    else:
        print("No new results found.")
else:
    print(f"❌❌❌ Error: Scenario file not found: {script_path} ❌❌❌")

# 🎛️ Custom Lab Bench (More Advanced. Doesn't Need To Be Used) 🎛️
### Tweak settings and run your own experiment. More Details at the Bottom.
###### WARNING 1: RAM related error? Run Step 2 again to re-initialize then choose a smaller model.
###### WARNING 2: Storage related error? Go to Runtime -> Disconnect and Delete Runtime. After restart, run Step 1.
###### WARNING 3: Rate Limit related error? Switch out the HuggingFace Token in Step 2 with a different one.

In [ ]:
import sys, os, glob, time
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import Image, Video, display

if not os.path.exists("main_configs"):
    if os.path.exists("Simplified-How-LLMs-Work-Visualized"):
        os.chdir("Simplified-How-LLMs-Work-Visualized")

if 'engine' not in locals():
    sys.path.append(os.path.abspath("main_configs"))
    sys.path.append(os.path.abspath("src"))
    try:
        import main as engine
    except ImportError:
        print("❌❌❌ Error: Could not import the engine. Did Cell 1 & 2 run successfully? ❌❌❌")

# --- MODEL CONTROLS --- #
#@markdown Choose the model to experiment with. The higher the "B" value, the larger and smarter the model is.
Model = "Mistral-7B" # @param ["GPT-2", "Pythia-160M", "Qwen2.5-0.5B", "TinyLlama-1.1B", "Qwen3-1.7B", "Phi-4-mini-4B", "Mistral-7B", "Qwen2.5-14B", "DeepSeek-Lite", "Qwen2.5-32B", "DeepSeek-R1", "Jamba-2-Mini", "Qwen2.5-72B", "Llama-4-Scout"]
#@markdown Choose the "System Prompt" that guides the model how it should behave.
Persona = "direct" # @param ["default", "direct", "caveman", "one-word", "angry", "nice", "liar", "biased", "pleaser", "insane", "sad"]

# --- ADVANCED SETTINGS --- #
#@markdown Keep 0.0 for AI's best predicted responses, a high value allows for the AI's responses to vary more.
Temperature = 0.0 # @param {type:"slider", min:0, max:5.0, step:0.1}
#@markdown Usually leave True to allow prompts to be structured. Turn off if using GPT-2/Pythia.
Use_Chat_Template = True # @param {type:"boolean"}
#@markdown Usually leave True to allow Persona to influence the sentiment_compass.
Sentiment_Use_Persona = True # @param {type:"boolean"}

# --- INPUTS --- #
# Prompt 1 is used for everything. Prompt 2 is ONLY used for the Comparison Video.
#@markdown The Main Prompt that all LLM models will use.
Prompt_1 = "Why do I feel the way I do?" # @param {type:"string"}
#@markdown The 2nd Prompt to compare with. Try to make the 2nd prompt similar to the Main Prompt.
Prompt_2 = "Why do I feel this way?" # @param {type:"string"}

# --- WHAT TO GENERATE? (Set to True/False) --- #
#@markdown See what the next token is that the model predicts.
Run_Prediction_Chart = True # @param {type:"boolean"}
#@markdown See what the model predicts token by token for a whole sequence.
Run_Sequence_Chart = True # @param {type:"boolean"}
#@markdown Compare 2 prompts to see how subtle changes alter model behavior, even when the core intent is identical.​
Run_Comparison_Video = True # @param {type:"boolean"}
#@markdown See how different models "Analyzes" tokens given a certain prompt. 
Run_Scan_Video = True # @param {type:"boolean"}
#@markdown See how the model considers generated token options that we consider to have emotional meaning.
Run_Sentiment_Compass = True # @param {type:"boolean"}

# --- APPLY SETTINGS --- #
engine.SELECTED_MODEL = Model
engine.CURRENT_PERSONA = Persona

# Apply Advanced Settings
engine.GENERATION_TEMPERATURE = Temperature
engine.USE_CHAT_TEMPLATE = Use_Chat_Template
engine.SENT_USE_PERSONA = Sentiment_Use_Persona

# Map Inputs #
engine.PRED_CHART_PROMPT = Prompt_1
engine.SEQ_CHART_PROMPT = Prompt_1
engine.SCAN_PROMPT = Prompt_1
engine.SENT_PROMPT = Prompt_1

# Map Comparison Inputs (Prompt 1 vs Prompt 2) #
engine.COMP_PROMPT_A = Prompt_1 
engine.COMP_PROMPT_B = Prompt_2 

engine.RUN_PREDICTION_CHART = Run_Prediction_Chart
engine.RUN_SEQUENCE_CHART = Run_Sequence_Chart
engine.RUN_COMPARISON_VIDEO = Run_Comparison_Video
engine.RUN_SCAN_VIDEO = Run_Scan_Video
engine.RUN_SENTIMENT_COMPASS = Run_Sentiment_Compass

engine.PRED_CHART_FILENAME = Path("my_prediction.png")
engine.SEQ_FILENAME = Path("my_sequence.png")

print(f"🧠 Loading {Model} with Persona: {Persona} (Temp: {Temperature})... 🧠")

# --- EXECUTE LOGIC --- #
try:
    start_time = time.time()
    engine.main()
    
    plt.close('all')
    print("\n--- RESULTS ---")
    
    files_root = glob.glob("*.png") + glob.glob("*.mp4")
    files_export = glob.glob("export/*.png") + glob.glob("export/*.mp4")
    all_files = files_root + files_export
    
    new_files = [f for f in all_files if os.path.getmtime(f) > start_time]
    
    if new_files:
        new_files.sort(key=lambda x: x.endswith(".png")) # Show videos first usually
        for f in new_files:
            print(f"Displaying: {os.path.basename(f)}")
            if f.endswith(".mp4"):
                display(Video(f, embed=True, width=825))
            else:
                display(Image(filename=f, width=825))
    else:
        print("No new output generated. (Make sure you checked a 'Run' box above!)")

except Exception as e:
    print(f"Error: {e}")
    print("If you are out of memory, try restarting the runtime.")

# 📚 Scenario Reference Guide 📚
Use this quick glossary to understand what each experiment demonstrates and what variables you might want to change. Learn more by reading the actual .py files in the scenarios_to_try directory/folder.

## 🧠 Reasoning & Logic 🧠
* **thinking_vs._chat_models**
    * *What it does:* Compares standard models against models that "think" (generate reasoning steps) before answering.
    * *Goal:* See how "thinking" tokens improve accuracy on math/logic problems.
* **simple_vs._complex_models**
    * *What it does:* Tests if models can solve trick logic puzzles where grammar suggests a math problem but logic dictates otherwise.
    * *Goal:* See if the model falls for the pattern or catches the trick.
* **raw_LLMS_vs._chatbot_LLMs**
    * *What it does:* Strips away the "Chatbot" formatting to reveal the raw text-completion engine underneath.
    * *Goal:* Watch the model get confused and try to autocomplete your question instead of answering it.

## ⚖️ Bias & Training Data ⚖️
* **bias_in_roles**
    * *What it does:* Tests for implicit gender bias in occupational pronouns (e.g., does it assume a Doctor is a "he"?).
    * *Goal:* Visualize how training data stereotypes bleed into AI predictions.
* **chinese_vs._USA_data**
    * *What it does:* Compares the knowledge base of Western models (Microsoft Phi) vs. Eastern models (Alibaba Qwen).
    * *Goal:* *Try switching the model* to see how culture affects "facts."
* **medical_bias**
    * *What it does:* Compares the safety advice given for Brand Names (Tylenol) vs. Chemical Names (Acetaminophen).
    * *Goal:* See how synonyms are treated as different ideas by the model.

## 🛑 Limitations & Failures 🛑
* **LLMs_suck**
    * *What it does:* Demonstrates "Tokenization Blindness" (e.g., why AI cannot count the letters in "Strawberry").
    * *Goal:* Understand that AI sees tokens (numbers), not letters.
* **knowledge_cutoff**
    * *What it does:* Asks about recent events vs. historical facts.
    * *Goal:* Visualize the "Frozen in Time" effect of static training data.
* **spelling_and_structure_matter!**
    * *What it does:* Compares the output quality of a prompt with typos vs. a perfect prompt.
    * *Goal:* See how "Garbage In" leads to "Garbage Out."

## 🛡️ Safety & Alignment 🛡️
* **safety_overrides**
    * *What it does:* Tests safety filters by comparing direct harmful requests vs. requests disguised as "creative writing" (Jailbreaking).
    * *Goal:* See where the safety boundary lies.
* **bad_prompts_vs._good_prompts**
    * *What it does:* Compares a vague prompt against a specific, constrained prompt.
    * *Goal:* Visualize how specific words narrow the probability space to get better results. 

# 🤖 AI Model Reference Guide 🤖
There are many different LLMs available. Here are a few common models used in the development and research community for running experiments but to also build actual tools and products. (Grouped by complexity).
Learn more about these models in model_manager.py

## 🔬 Research & Legacy Tier (The Ancestors) 🔬
*(Smaller or old research models. Can be run on old laptops. Focus more on raw patterns).*
* **GPT-2 (124M)**
    * A very small, older model (2019) from OpenAI.
* **Pythia-160M**
    * Designed for scientific research on how models learn that use fundamental LLM logic.

## 📱 Lightweight Tier (Laptop and Mobile Friendly) 📱
*Small models that are surprisingly smart despite parameter size*
* **TinyLlama-1.1B**
    * Same architecture and tokenizer as Meta's Llama 2 models. Popular for niche cases and precise systems.
* **Qwen3-1.7B**
    * Smaller design based on larger Alibaba model. Focuses more on basic coding and reasoning.
* **Phi-4-mini-4B** (⭐ Recommended Default)
    * Derivative of the 14B variation, a model that approaches Gemini 1.5 Flash and even GPT o1-mini & GPT-4o in  precise tasks.

## 🧠 Medium Weight Tier (Local Desktop Models) 🧠
*Strong models that only require a single decent GPU that nearly anyone can buy.*
* **Mistral-7B**
    * Comparable and in many cases, better than early GPT-3.5 in general capabilities.
* **Qwen2.5-14B**
    * Weaker reasoning than "deepseek-ai/DeepSeek-V2-Lite-Chat", but overall still good general abilities. In terms of performance, it's a lot closer to GPT-3.5 performance overall than Mistral-7B.
* **DeepSeek-Lite (16B)**
    * Cutting edge model from China exploring new ways to develop LLM based models. Positioned between advanced models like Claude 3.5 Sonnet and o1-mini in coding and reasoning applications. Can be run on a single 2080 Ti.

## 🚀 Heavyweight & More Modern Models Tier 🚀
*More Cutting-edge models. May crash on smaller Google Colab instances. Can be run on very strong GPUs or dual to quad GPU setup in a Desktop PC.*
* **Qwen2.5-32B**
    * When tuned well, can be comparable to GPT-4 to GPT-4o performance on various benchmarks.
* **DeepSeek-R1 (Distill) (32B)**
    * Consistently competitive against OpenAI-o1-mini (GPT o1-mini).
* **Jamba-2-Mini (12B Active | 52B Total)**
    * A hybrid experimental design (Mamba architecture) released in 2026. Claims it is comparable and even better vs. original GPT-4, even GPT-4o in some cases.

## ⭐ Very Modern But Very Large. Barely Fits in a Google Colab Pro Environment ⭐
*Cutting-edge models. Likely crash on smaller Google Colab instances or ones that have space being used already.*
* **Qwen2.5-72B**
    * Considered to be stronger than GPT-3.5 & GPT-4, even GPT-4 Turbo in some cases.
* **Llama-4-Scout**
    * Example of a "small" modern model at the cutting edge. 109B Parameters (17B+ Active). 200+ GB of VRAM. A very modern and cutting edge model made by Meta/FaceBook. (Likely need to share contact info).
    Essentially the same performance, if not, very similar performance to GPT-4 and GPT-4o in most applications.

# 🎭 Persona Reference Guide 🎭
Use this list to give the AI a specific personality or set of constraints. Changing the persona changes the system instructions hidden from the user. Learn more by viewing the personas.yaml file in the folder main_configs.

## 🛠️ Utility & Control 🛠️
* **default**
    * *Description:* The raw model behavior with no added instructions.
    * *Best For:* Seeing the "base" model's true nature.
* **direct**
    * *Description:* Try to force the AI to be concise. It strips away "As an AI language model..." filler.
    * *Best For:* Getting straight answers without lectures.
* **secret**
    * *Description:* A persona that strictly believes it is a human and denies being a computer.
    * *Best For:* The "Server Prompting" scenario.

## 🧱 Constraints & Grammar 🧱
* **caveman**
    * *Description:* Try to force the model to use broken grammar and capitalization.
    * *Best For:* Testing if an AI can be "dumbed down" or if it reverts to proper English.
* **one-word**
    * *Description:* An extreme constraint that tries to force the model to answer in exactly one word.
    * *Best For:* Testing how well a model follows strict negative constraints.

## ❤️ Emotions (Sentiment Steering) ❤️
* **nice**
    * *Description:* The "Best Friend." Extremely supportive, happy, and positive.
    * *Best For:* Try to force the shifting of the Sentiment Compass to the Top-Right (Active/Positive).
* **sad**
    * *Description:* Depressed, low energy, and uses lowercase text.
    * *Best For:* Try to force the shifting of the Sentiment Compass to the Bottom-Left (Passive/Negative).
* **angry**
    * *Description:* Hostile, rude, and aggressive.
    * *Best For:* Try to force the shifting of the Sentiment Compass to the Top-Left (Active/Negative).

## 😈 Adversarial & Failures 😈
* **liar**
    * *Description:* Instructed to always try to provide false information.
    * *Best For:* Visualizing hallucinations and fact-checking failures.
* **pleaser**
    * *Description:* Instructed to try to agree with everything the user says, even if it is wrong (Sycophancy).
    * *Best For:* Seeing how AI reinforces user bias.
* **biased**
    * *Description:* A stubborn persona that tries to disagree with everything and holds incorrect opinions.
    * *Best For:* Testing the model's ability to reason against its own training data.
* **insane**
    * *Description:* Outputs chaotic, nonsensical text.
    * *Best For:* High entropy visualization.